# Padronização dos dados com o Portal Brasileiro de Dados Abertos (CGU)
### Limpeza da lista de instituições e construção da correspondência entre temas da CGU e da Câmara dos Deputados

Queremos padronizar o que for possível dos nossos dados com os dados daqui: <https://dados.gov.br/dados/reusos>. 

In [1]:
import json

## Functions

In [2]:
def load_json(filename):
    
    with open(filename, 'r') as f:
        data = json.load(f)
    
    return data

## ETL institution data

In [3]:
from glob import glob
from collections import defaultdict

In [4]:
# Load institution names:
inst_files = glob('../dados/aux/organizacao_*.json')
institutions = []
ids = []
for f in inst_files:
    institutions += [(i['titulo'], i['id']) for i in load_json(f)]
# Sort list:
institutions = sorted(institutions)
# Create translator (name > ID):
inst2id = defaultdict(lambda: None, dict(institutions))

### Checks

In [5]:
len(institutions) == len(set(institutions))

True

### List

In [6]:
for a in [i[0] for i in institutions][:-27]:
    print(a)

Advocacia-Geral da União - AGU
Agência Espacial Brasileira - AEB
Agência Nacional de Aviação Civil - ANAC
Agência Nacional de Energia Elétrica - ANEEL
Agência Nacional de Mineração
Agência Nacional de Saúde Suplementar
Agência Nacional de Telecomunicações
Agência Nacional de Transportes Aquaviários
Agência Nacional de Transportes Terrestres
Agência Nacional de Vigilância Sanitária - ANVISA
Agência Nacional de Águas e Saneamento Básico
Agência Nacional do Cinema
Agência Nacional do Petróleo, Gás Natural e Biocombustíveis - ANP
Arquivo Nacional - AN
Assembleia Legislativa de Minas Gerais - ALMG
Casa Civil da Presidência da República
Centro Federal de Educação Tecnológica - Celso Suckow da Fonseca - CEFET-RJ
Centro Federal de Educação Tecnológica de Minas Gerais - CEFET-MG
Centro de Tecnologia Mineral
Centro de Tecnologia da Informação Renato Archer
Comissão de Valores Mobiliários
Conselho Administrativo de Defesa Econômica - Cade
Conselho Nacional de Desenvolvimento Científico e Tecnológ

Organizações que não tem o nome human-friendly:

In [7]:
for a in [i[0] for i in institutions][-27:]:
    print(a)

banco-central-do-brasil-bcb
banco-nacional-de-desenvolvimento-economico-e-social
caixa-economica-federal-cef
camara-municipal-de-comodoro
colegio-pedro-ii-cpii
coordenacao-de-aperfeicoamento-de-pessoal-de-nivel-superior-capes
ebserh-filial-hospital-universitario-de-santa-maria
estado-de-alagoas-al
estado-de-minas-gerais-mg
exercito-brasileiro
fundacao-casa-de-rui-barbosa-fcrb
gsi
hu-ufscar
instituto-federal-de-educacao-ciencia-e-tecnologia-do-amazonas-ifam
instituto-federal-de-educacao-ciencia-e-tecnologia-do-rio-grande-do-sul-ifrs
instituto-nacional-de-estudos-e-pesquisas-educacionais-anisio-teixeira-inep
ipea
ministerio-da-infraestrutura-minfra
ministerio-da-previdencia-social
mjsp
museu-de-astronomia-e-ciencias-afins-mast
prefeitura-municipal-de-mogi-das-cruzes
secretaria-geral-sgpr
superintendencia-do-desenvolvimento-do-nordeste-sudene
tribunal-de-contas-da-uniao-tcu
universidade-federal-de-sao-paulo
universidade-federal-goias


## ETL topic data

In [8]:
# CGU topics:
cgu_topics_raw = load_json('../dados/aux/temas_cgu_v1.json')
cgu_topics = [c['title'] for c in cgu_topics_raw]

In [8]:
# CORDATA v01 topics:
with open('../dados/aux/temas_cordata_v1.json', 'r') as f:
    cordata_topics = [line.rstrip() for line in f]

In [15]:
# Manually created translation dict:
cordata2cgu = {'Agropecuária':'Agricultura, extrativismo e pesca',
               'Administração Pública':'Governo e Política',
               'Alimentação':'Assistência e Desenvolvimento Social',
               'Arte e Cultura':'Cultura, Lazer e Esporte',
               'Atividade industrial, de comércio ou serviços':'Comércio, Serviços e Turismo',
               'Ciência e Tecnologia':'Ciência, Informação e Comunicação',
               'Comunicações':'Ciência, Informação e Comunicação',
               'Consumo':'Economia e Finanças',
               'Dados demográficos':'Dados Estratégicos',
               'Defesa':'Defesa e Segurança',
               'Direito e Processual Penal':'Justiça e Legislação',
               'Direitos Humanos':'Assistência e Desenvolvimento Social',
               'Economia':'Economia e Finanças',
               'Esporte':'Cultura, Lazer e Esporte',
               'Finanças e Orçamento Público':'Economia e Finanças',
               'Imóveis, Habitação e Urbanismo':'Habitação, Saneamento e Urbanismo',
               'Justiça e Direito':'Justiça e Legislação',
               'Lazer':'Cultura, Lazer e Esporte',
               'Multimídia':'Cultura, Lazer e Esporte',
               'Política, Partidos e Eleições':'Governo e Política',
               'Política, Partidos e Eleições': 'Governo e Política',
                'Previdência e Assistência Social': 'Assistência e Desenvolvimento Social',
                'Processo Legislativo': 'Governo e Política',
                'Recursos hídricos': 'Meio Ambiente',
                'Recursos minerais': 'Economia e Finanças',
                'Redes Sociais': 'Ciência, Informação e Comunicação',
                'Religião': 'Cultura, Lazer e Esporte',
                'Segurança': 'Defesa e Segurança',
                'Terras': 'Geografia',
                'Trabalho e Emprego': 'Trabalho',
                'Transparência': 'Governo e Política',
                'Transporte': 'Transportes e Trânsito',
                'Turismo': 'Comércio, Serviços e Turismo'
               }

In [89]:
len(cgu_topics)

22

In [45]:
for t in cgu_topics:
    print(t)

Agricultura, extrativismo e pesca
Assistência e Desenvolvimento Social
Ciência, Informação e Comunicação
Comércio, Serviços e Turismo
Cultura, Lazer e Esporte
Dados Estratégicos
Defesa e Segurança
Economia e Finanças
Educação
Energia
Equipamentos Públicos
Geografia
Governo e Política
Habitação, Saneamento e Urbanismo
Indústria
Justiça e Legislação
Meio Ambiente
Plano Plurianual
Relações Internacionais
Saúde
Trabalho
Transportes e Trânsito


### List all topics

In [53]:
import pandas as pd

import clean as cc

In [54]:
# Load translations:
translation_df = pd.read_excel('../docs/conteudo_site/textos_site_traducao_traduzidos_v04.xlsx', sheet_name='Site')

# Build translators:
tema_map = cc.get_translator(translation_df, r'filtro_tema_\d')

In [59]:
def wrap(topics):
    usecase = {'topics': topics}
    data = [usecase]
    meta = {'data':data}
    return meta

In [60]:
# Save to file:
if True:
    with open('/home/hxavier/temp/cgu_topics.json', 'w') as f:
        json.dump(wrap(cgu_topics), f, ensure_ascii=False, indent=2)

In [61]:
es = [tema_map[t] for t in cgu_topics]

In [62]:
# Save to file:
if True:
    with open('/home/hxavier/temp/cgu_topics_es.json', 'w') as f:
        json.dump(wrap(es), f, ensure_ascii=False, indent=2)

In [64]:
# Check if JSON is correct after manual modification:
check_ok = load_json('../dados/limpos/usecases_n12_2024-01-02.json')

### Checks

In [19]:
set(cordata_topics) - set(cordata2cgu.keys())

{'Educação', 'Energia', 'Meio Ambiente', 'Relações Internacionais', 'Saúde'}

In [24]:
set(cgu_topics) - set(cordata2cgu.values())

{'Educação',
 'Energia',
 'Equipamentos Públicos',
 'Indústria',
 'Plano Plurianual',
 'Relações Internacionais',
 'Saúde'}

In [23]:
set(cordata2cgu.values()) - set(cgu_topics)

set()

In [27]:
extra_trad = sorted(list(set(cgu_topics) - set(cordata_topics)))

### Exports

In [31]:
# For exporting to Excel for translation:
for t in extra_trad:
    print(t)

Agricultura, extrativismo e pesca
Assistência e Desenvolvimento Social
Ciência, Informação e Comunicação
Comércio, Serviços e Turismo
Cultura, Lazer e Esporte
Dados Estratégicos
Defesa e Segurança
Economia e Finanças
Equipamentos Públicos
Geografia
Governo e Política
Habitação, Saneamento e Urbanismo
Indústria
Justiça e Legislação
Plano Plurianual
Trabalho
Transportes e Trânsito
